# Azure ML Workspace

## Importing libraries

In [ ]:
from azureml.core import Workspace
from azureml.core.environment import Environment

## Initialise workspace
Initialise a workspace using the values from config.json
This Json file was downloaded after provisioning a workspace through the online interface.
This could also be created programmatically using Workspace.create()

In [ ]:
workspace = Workspace.from_config()

## Register the ML model
Load the ML model into the workspace from file.
This file was produced previously.
This method will allow different models to be loaded using the same workspace.

In [ ]:
##  Load the ML model from file. This file will most likely be a pickle dump.

## Create an environment
Create an environment for the model to run on the Azure servers.

In [ ]:
environment = Environment('env')
##  Load conda dependencies required for this environment.
##  Register environment for the workspace

## Provision the hardware to be used
Define the hardware configuration that will be used to process the model.

In [ ]:
##  Define the hardware config and deployment config